# Deep Nets with TF Abstractions


# The Data

In [ ]:
#from sklearn.datasets import load_wine
import pandas as pd
import numpy

In [ ]:
dataset = pd.read_csv("/content/Ina_New.csv")

feat_data = dataset[['C', 'Cr', 'Mn', 'Ni', 'Si', 'CR']].values
labels = dataset['Hardness'].values

In [ ]:
a = numpy.array(labels, dtype=numpy.float64)
labels = a.astype(numpy.int64)
k = set()

for l in labels:
  k.add(l)

len(list(k))


657

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

### Scale the Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaled_x_train = scaler.fit_transform(X_train)

In [ ]:
scaled_x_test = scaler.transform(X_test)

# Abstractions

## Estimator API

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import estimator 

In [ ]:
X_train.shape

(6977, 6)

In [ ]:
feat_cols = [tf.feature_column.numeric_column("x", shape=[6])]

In [ ]:
deep_model = estimator.DNNClassifier(hidden_units=[13, 13, 13],
                            feature_columns=feat_cols,
                            n_classes=1077,
                            optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01) )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpdtr481a6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbbeaf2bc88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},y=y_train,shuffle=True,batch_size=10,num_epochs=5)

In [ ]:
print(y_train, scaled_x_train)

[283 563 564 ... 531 127 365] [[0.         0.         0.         0.5        0.33333333 0.3877551 ]
 [0.42857143 0.         0.31034483 0.5        1.         0.12244898]
 [1.         0.         0.         1.         0.33333333 0.12244898]
 ...
 [1.         1.         0.31034483 0.         0.         0.79591837]
 [0.         1.         0.31034483 1.         0.         0.06122449]
 [0.         0.1        0.31034483 0.5        0.33333333 0.14285714]]


In [ ]:
deep_model.train(input_fn=input_fn,steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpdtr481a6/model.ckpt.
INFO:tensorflow:loss = 69.971176, step = 1
INFO:tensorflow:global_step/sec: 462.206
INFO:tensorflow:loss = 69.611664, step = 101 (0.220 sec)
INFO:tensorflow:global_step/sec: 612.634
INFO:tensorflow:loss = 69.73984, step = 201 (0.161 sec)
INFO:tensorflow:global_step/sec: 570.242
INFO:tensorflow:loss = 61.4236, step = 301 (0.176 sec)
INFO:tensorflow:global_step/sec: 633.604
INFO:tensorflow:loss = 64.00861, step = 401 (0.157 sec)
INFO:tensorflow:Saving checkpoints for 500 into /tmp/tmpdtr481a6/model.ckpt.
INFO:tensorflow:Loss for final step: 60.058613.


In [ ]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},shuffle=False)

In [ ]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpdtr481a6/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
predictions = [p['class_ids'][0] for p in preds]

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         104       0.00      0.00      0.00         6
         105       0.00      0.00      0.00         5
         108       0.00      0.00      0.00         9
         109       0.00      0.00      0.00        11
         111       0.00      0.00      0.00         4
         112       0.00      0.00      0.00        10
         113       0.00      0.00      0.00         5
         115       0.00      0.00      0.00         4
         116       0.00      0.00      0.00         6
         117       0.00      0.00      0.00        21
         120       0.00      0.00      0.00        20
         121       0.00      0.00      0.00         7
         123       0.00      0.00      0.00        12
         126       0.00      0.00      0.00         6
         127       0.00      0.00      0.00         2
         128       0.00      0.00      0.00         5
         130       0.00      0.00      0.00         3
         131       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


____________
______________

# TensorFlow Keras

### Create the Model

In [ ]:
from tensorflow.contrib.keras import models

In [ ]:
dnn_keras_model = models.Sequential()

### Add Layers to the model

In [ ]:
from tensorflow.contrib.keras import layers

In [ ]:
dnn_keras_model.add(layers.Dense(units=6,input_dim=6,activation='relu'))

In [ ]:
dnn_keras_model.add(layers.Dense(units=6,activation='relu'))
dnn_keras_model.add(layers.Dense(units=6,activation='relu'))

In [ ]:
dnn_keras_model.add(layers.Dense(units=850,activation='softmax'))

### Compile the Model

In [ ]:
from tensorflow.contrib.keras import losses,optimizers,metrics

In [ ]:
# explore these
# losses.

In [ ]:
#optimizers.

In [ ]:
losses.sparse_categorical_crossentropy

<function tensorflow.python.keras.losses.sparse_categorical_crossentropy>

In [ ]:
dnn_keras_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



```
# This is formatted as code
```

### Train Model

In [ ]:
y_train

array([283, 563, 564, ..., 531, 127, 365])

In [ ]:
dnn_keras_model.fit(scaled_x_train,y_train,epochs=50)

Train on 6977 samples
Epoch 1/50
6977/6977 [==============================] - 2s 298us/sample - loss: 6.6670 - acc: 0.0050
Epoch 2/50
6977/6977 [==============================] - 2s 288us/sample - loss: 6.5236 - acc: 0.0057
Epoch 3/50
6977/6977 [==============================] - 2s 297us/sample - loss: 6.4102 - acc: 0.0066
Epoch 4/50
6977/6977 [==============================] - 2s 293us/sample - loss: 6.3074 - acc: 0.0085
Epoch 5/50
6977/6977 [==============================] - 2s 290us/sample - loss: 6.2270 - acc: 0.0085
Epoch 6/50
6977/6977 [==============================] - 2s 289us/sample - loss: 6.1731 - acc: 0.0085
Epoch 7/50
6977/6977 [==============================] - 2s 294us/sample - loss: 6.1373 - acc: 0.0085
Epoch 8/50
6977/6977 [==============================] - 2s 286us/sample - loss: 6.1120 - acc: 0.0085
Epoch 9/50
6977/6977 [==============================] - 2s 287us/sample - loss: 6.0938 - acc: 0.0085
Epoch 10/50
6977/6977 [==============================] - 2s 285us/sam

In [ ]:
predictions = dnn_keras_model.predict_classes(scaled_x_test)

In [ ]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

         104       0.00      0.00      0.00         0
         105       0.00      0.00      0.00         0
         108       0.00      0.00      0.00         0
         109       0.00      0.00      0.00         0
         111       0.00      0.00      0.00         0
         112       0.00      0.00      0.00         0
         113       0.00      0.00      0.00         0
         115       0.00      0.00      0.00         0
         116       0.00      0.00      0.00         0
         117       0.00      0.00      0.00         0
         120       0.00      0.00      0.00         0
         121       0.00      0.00      0.00         0
         123       0.00      0.00      0.00         0
         126       0.00      0.00      0.00         0
         127       0.00      0.00      0.00         0
         128       0.00      0.00      0.00         0
         130       0.00      0.00      0.00         0
         131       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Layers API

https://www.tensorflow.org/tutorials/layers

## Formating Data

In [ ]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
wine_data = load_wine()
feat_data = wine_data['data']
labels = wine_data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

In [ ]:
scaler = MinMaxScaler()
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)
# ONE HOT ENCODED

onehot_y_train = pd.get_dummies(y_train).values
one_hot_y_test = pd.get_dummies(y_test).values

### Parameters

In [ ]:
num_feat = 124
num_hidden1 = 124
num_hidden2 = 124
num_outputs = 13
learning_rate = 0.01

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

### Placeholder

In [ ]:
X = tf.placeholder(tf.float32,shape=[124,13])
y_true = tf.placeholder(tf.float32,shape=[124,13])

### Activation Function

In [ ]:
actf = tf.nn.relu

### Create Layers

In [ ]:
hidden1 = fully_connected(X,num_hidden1,activation_fn=actf)

In [ ]:
hidden2 = fully_connected(hidden1,num_hidden2,activation_fn=actf)

In [ ]:
output = fully_connected(hidden2,num_outputs)

### Loss Function

In [ ]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits=output)

### Optimizer

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

### Init

In [2]:
init = tf.global_variables_initializer()

NameError: ignored

In [1]:
training_steps = 1000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(training_steps):
        sess.run(train,feed_dict={X:scaled_x_train,y_true:y_train})
        
    # Get Predictions
    logits = output.eval(feed_dict={X:scaled_x_test})
    
    preds = tf.argmax(logits,axis=1)
    
    results = preds.eval()

NameError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(results,y_test))

In [ ]:


#feat_data = dataset[['C', 'Cr', 'Mn', 'Ni', 'Si', 'CR']].values
